In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from ex17_1_lib import DNLDS, EKF, SUT, SPKF

In [ ]:
# Autonomous pendulum
# x = [theta, omega, L, alfa]
n = 4 
f = lambda x, t, u: np.array([ x[1], -9.81/x[2]*np.sin(x[0])-x[3]*x[1], 0, 0 ])
h = lambda x: np.array([x[0]])
dfdx = lambda x, u: np.array([ [0, 1, 0, 0],
                               [-9.81/x[2]*np.cos(x[0]), -x[3], +9.81*np.sin(x[0])/(x[2]**2), -x[1] ],
                               [0 , 0 , 0, 0],
                               [0, 0, 0, 0]]) # Jacobian of f2
dhdx = lambda x: np.array([[1, 0, 0, 0]]) # Jacobian of h2
G = np.eye(n)
Q = np.diag([0, 0, 0, 0])
R = np.diag([0.04*0.04])
x0 = np.array([10.0/180*math.pi,0,1.3,0.2])
x0_est = np.array([12.0/180*math.pi,0,1,0.1])
P0 = np.diag([1,0.1*0.1,3*3,0.15**2])

In [ ]:
# Init Dynamical System
nls = DNLDS(f, h, G, Q, R, x0)

# Init Kalman Filter
Q = Q + np.diag([10**-4, 10**-4, 10**-9, 10**-9])

# EKF (sometimes it diverges really bad on this model, like 1 out 5 is bad; model is also highly non-linear)
# filter = EKF(f, h, dfdx, dhdx, G, Q, R, x0_est, P0)  

# OR Sigma Point Kalman Filter ( better on this pendulum sistem )
alpha = 0.0001
kappa = 0.0
beta = 2.0
sut = SUT(alpha, beta, kappa, n)
filter = SPKF(f,h,G,Q,R,x0_est,P0,sut,variant=0)  # 0 - normal UKF, 1 - IUKF, 2 - UKFz

In [ ]:
# Logging 
x_true = []; x_true.append(nls.x)
x_est = []; x_est.append(filter.x)

T = 10 # in seconds 
dt = 0.01 # seconds 

for i in range(int(T/dt)):
    
    nls.step(0,dt,1)
    filter.predict(0,dt,1)
     
    meas = nls.output()
    filter.update(meas,1) # 0 - simple covariance update, 1 - Joseph covariance update

    # Logging 
    x_true.append(nls.x)
    x_est.append(filter.x)

x_true = np.array(x_true)
x_est = np.array(x_est)

In [ ]:
# Plot the logs
for i in range(nls.x.shape[0]):
    _, ax = plt.subplots(1)
    plt.style.use('seaborn-whitegrid')
    ax.plot(x_true[:,i],color='blue', linestyle='solid', marker='o',
        markerfacecolor='blue', markersize=4, label='True State')
    ax.plot(x_est[:,i],color='orange', linestyle='solid', marker='o',
        markerfacecolor='orange', markersize=4, label='Est State')
    ax.set_title('State '+str(i))
    plt.legend()